## **<h3 align="center"> Computational Intelligence for Optimization</h3>**
# **<h3 align="center">Testing Mutations Operators</h3>**
**Group members:**<br>
Alexandra Pinto - 20211599@novaims.unl.pt - 20211599<br>
Julia Karpienia  - 20240514@novaims.unl.pt - 20240514<br>
Steven Carlson - 20240554@novaims.unl.pt - 20240554 <br>
Tim Straub - 20240505@novaims.unl.pt - 20240505

<a id = "toc"></a>

## Table of Contents

1. [Introduction](#intro)
2. [Import & Setup](#import_setup)
3. [Generate Initial Population](#gen_initial_pop)
4. [Define General Mutators Testing Function](#testing_mutators_function)
5. [Run Mutators Tests](#run_mutators_test)
   - 5.1. [Single Player Swap](#single_player_swap)
   - 5.2. [Circular Position Shift](#circular_pos_shift)
   - 5.3. [Full Position Swap](#full_pos_swap)
6. [Conclusion](#conclusion)


#  1. Introduction <a class="anchor" id="intro"></a>
[Back to ToC](#toc)<br>

This notebook is used to **test and understand how each mutator operator behaves**. We run multiple trials to evaluate if the offspring generated are valid, different from their parents, and follow the intended logic.

We also added **print statements inside some of the mutator functions** to help visualize the process and debug. These prints will be **commented out later** to avoid excessive output during the final Grid Search and evaluation phases.


# 2. Import & Setup <a name="import_setup"></a>

[Back to ToC](#toc)<br>


In [ ]:
# From the Operators folder, import all the necessary modules
# To create a new population, we need to import the following:
from Operators.population import *
# To call the mutators, we need to import the following:
from Operators.mutations import *

import random
from collections import Counter
from copy import deepcopy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload



# 3. Generate Initial Population <a class="anchor" id="gen_initial_pop"></a>

[Back to ToC](#toc)<br>

In this section, we use the functions `load_players_from_csv` and `generate_population` from `population.py` to create the initial population. This population will serve as the basis for testing the mutation operators in the following sections.

In [ ]:
players = load_players_from_csv("data/players(in).csv")
players

[Alex Carter (GK) Skill: 85 Salary: €90.0M,
 Jordan Smith (GK) Skill: 88 Salary: €100.0M,
 Ryan Mitchell (GK) Skill: 83 Salary: €85.0M,
 Chris Thompson (GK) Skill: 80 Salary: €80.0M,
 Blake Henderson (GK) Skill: 87 Salary: €95.0M,
 Daniel Foster (DEF) Skill: 90 Salary: €110.0M,
 Lucas Bennett (DEF) Skill: 85 Salary: €90.0M,
 Owen Parker (DEF) Skill: 88 Salary: €100.0M,
 Ethan Howard (DEF) Skill: 80 Salary: €70.0M,
 Mason Reed (DEF) Skill: 82 Salary: €75.0M,
 Logan Brooks (DEF) Skill: 86 Salary: €95.0M,
 Caleb Fisher (DEF) Skill: 84 Salary: €85.0M,
 Nathan Wright (MID) Skill: 92 Salary: €120.0M,
 Connor Hayes (MID) Skill: 89 Salary: €105.0M,
 Dylan Morgan (MID) Skill: 91 Salary: €115.0M,
 Hunter Cooper (MID) Skill: 83 Salary: €85.0M,
 Austin Torres (MID) Skill: 82 Salary: €80.0M,
 Gavin Richardson (MID) Skill: 87 Salary: €95.0M,
 Spencer Ward (MID) Skill: 84 Salary: €85.0M,
 Sebastian Perry (FWD) Skill: 95 Salary: €150.0M,
 Xavier Bryant (FWD) Skill: 90 Salary: €120.0M,
 Elijah Sanders 

Let's generate 10 leagues.

In [5]:
population = generate_population(players, num_leagues=10)

for i, league in enumerate(population):
    print(f"\n--- League {i+1} ---")
    print(league)
    print(f"Standard Deviation of Avg Skills: {league.get_skill_std_dev():.2f}")


--- League 1 ---
Jordan Smith (GK) - Skill: 88, Cost: 100.0M
Lucas Bennett (DEF) - Skill: 85, Cost: 90.0M
Caleb Fisher (DEF) - Skill: 84, Cost: 85.0M
Spencer Ward (MID) - Skill: 84, Cost: 85.0M
Dominic Bell (MID) - Skill: 86, Cost: 95.0M
Chase Murphy (FWD) - Skill: 86, Cost: 95.0M
Landon Powell (FWD) - Skill: 89, Cost: 110.0M

Alex Carter (GK) - Skill: 85, Cost: 90.0M
Brayden Hughes (DEF) - Skill: 87, Cost: 100.0M
Maxwell Flores (DEF) - Skill: 81, Cost: 72.0M
Gavin Richardson (MID) - Skill: 87, Cost: 95.0M
Ashton Phillips (MID) - Skill: 90, Cost: 110.0M
Julian Scott (FWD) - Skill: 92, Cost: 130.0M
Colton Gray (FWD) - Skill: 91, Cost: 125.0M

Ryan Mitchell (GK) - Skill: 83, Cost: 85.0M
Mason Reed (DEF) - Skill: 82, Cost: 75.0M
Logan Brooks (DEF) - Skill: 86, Cost: 95.0M
Hunter Cooper (MID) - Skill: 83, Cost: 85.0M
Bentley Rivera (MID) - Skill: 88, Cost: 100.0M
Tyler Jenkins (FWD) - Skill: 80, Cost: 70.0M
Sebastian Perry (FWD) - Skill: 95, Cost: 150.0M

Blake Henderson (GK) - Skill: 87,

# 4. Define General Mutators Testing Function <a class="anchor" id="testing_mutators_function"></a>


[Back to ToC](#toc)<br>
Let's run a series of randomized trials using the `test_mutation` function to assess:

- Whether the mutation produces valid children. (valid mutation)
- How often the operation has no effect (no-op (unchanged)).
- If any invalid outputs are generated, taking into account the constraints.  invalid (None)

In [ ]:
def test_mutation(population, mutation_fn, mutation_name, mut_prob, trials=100):
    """
    Tests a mutation operator by applying it to each individual in the population
    multiple times and reporting statistics on:
      - valid mutations (new, non-None, changed)
      - invalid mutations (returned None)
      - no-op mutations (returned copy identical to parent)
    """
    overall_stats = Counter()

    for idx, original in enumerate(population):
        orig_str = str(original)
        stats = Counter()
        for _ in range(trials):
            mutated = mutation_fn(original, mut_prob)
            if mutated is None:
                stats['invalid (None)'] += 1
            else:
                # Compare string representations to detect change/no-op
                if str(mutated) == orig_str:
                    stats['no-op (unchanged)'] += 1
                else:
                    stats['valid mutation'] += 1
        # Print per-individual stats
        print(f"{mutation_name} – League #{idx} over {trials} trials:")
        for k, v in stats.items():
            print(f"  {k}: {v}")
        print()
        overall_stats.update(stats)

    # Print overall summary
    print(f"Overall {mutation_name} stats across population ({len(population)} leagues × {trials} trials):")
    total = sum(overall_stats.values())
    for k, v in overall_stats.items():
        pct = v / total * 100
        print(f"  {k}: {v} ({pct:.1f}%)")


# 5. Run Mutators Tests <a class="anchor" id="run_mutators_test"></a>
[Back to ToC](#toc)<br>

## 5.1. Single Player Swap <a class="anchor" id="single_player_swap"></a>
[Back to 5. Run Mutators Tests](#run_mutators_test)<br>

This mutation selects two different teams and swaps one player of the same position between them. It introduces light structural diversity while ensuring feasibility and preserving team balance.

**Steps:**

1. Select two distinct teams from the league.

2. Randomly choose a position (GK, DEF, MID, FWD).

3. Randomly pick one player from each team in the chosen position.

4. Swap the players between the two teams.

Validate both teams and the new league. If invalid, discard mutation.

**Example:**

- Team A has: [GK1, DEF1, MID1, FWD1]

- Team B has: [GK2, DEF2, MID2, FWD2]

If the position chosen is MID, we might swap MID1 with MID2, resulting in small but effective diversity.

This mutation is conservative and ideal for fine-tuning team compositions.

In [8]:
# Test Swap Mutation
test_mutation(
    population,
    single_player_swap_2teams,
    mutation_name="Swap Mutation",
    mut_prob=0.5,
    trials=50
)

Swap Mutation – League #0 over 50 trials:
  valid mutation: 27
  no-op (unchanged): 23

Swap Mutation – League #1 over 50 trials:
  valid mutation: 22
  no-op (unchanged): 28

Swap Mutation – League #2 over 50 trials:
  no-op (unchanged): 23
  valid mutation: 27

Swap Mutation – League #3 over 50 trials:
  no-op (unchanged): 32
  valid mutation: 18

Invalid mutation: returning NONE
Swap Mutation – League #4 over 50 trials:
  no-op (unchanged): 25
  valid mutation: 24
  invalid (None): 1

Invalid mutation: returning NONE
Invalid mutation: returning NONE
Swap Mutation – League #5 over 50 trials:
  valid mutation: 22
  no-op (unchanged): 26
  invalid (None): 2

Invalid mutation: returning NONE
Swap Mutation – League #6 over 50 trials:
  no-op (unchanged): 20
  valid mutation: 29
  invalid (None): 1

Invalid mutation: returning NONE
Swap Mutation – League #7 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 25
  invalid (None): 1

Invalid mutation: returning NONE
Invalid mutation: 


## 5.2. Circular Position Shift <a class="anchor" id="circular_pos_shift"></a>
[Back to 5. Run Mutators Tests](#run_mutators_test)<br>

This mutation selects a single position and shifts one player of that position from each team to the next team in a circular manner. It helps propagate talent evenly and explore more global solutions.

**Steps:**

1. Randomly select a position (GK, DEF, MID, FWD).

2. Pick one player from each team in that position.

3. Perform a circular shift:
    Each team gives their selected player to the next team (last team gives to the first).

Validate all teams and the league.

**Example:**

- Teams: [T1, T2, T3]

- Chosen position: DEF

- Selected: DEF1 (T1), DEF2 (T2), DEF3 (T3)

After mutation:

- T1 gets DEF3, T2 gets DEF1, T3 gets DEF2

This operator introduces coordinated changes across all teams, ideal for global exploration.

In [9]:
test_mutation(
    population,
    single_player_shift_all_teams,
    mutation_name="Shift Mutation",
    mut_prob=0.5,
    trials=50
)

Shift Mutation – League #0 over 50 trials:
  valid mutation: 20
  invalid (None): 3
  no-op (unchanged): 27

Shift Mutation – League #1 over 50 trials:
  valid mutation: 28
  no-op (unchanged): 20
  invalid (None): 2

Shift Mutation – League #2 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 25
  invalid (None): 1

Shift Mutation – League #3 over 50 trials:
  valid mutation: 20
  no-op (unchanged): 28
  invalid (None): 2

Shift Mutation – League #4 over 50 trials:
  valid mutation: 23
  no-op (unchanged): 26
  invalid (None): 1

Shift Mutation – League #5 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 22
  invalid (None): 4

Shift Mutation – League #6 over 50 trials:
  valid mutation: 24
  no-op (unchanged): 24
  invalid (None): 2

Shift Mutation – League #7 over 50 trials:
  no-op (unchanged): 21
  valid mutation: 29

Shift Mutation – League #8 over 50 trials:
  valid mutation: 27
  no-op (unchanged): 20
  invalid (None): 3

Shift Mutation – League #9 over 50 tria


## 5.3. Full Position Swap <a class="anchor" id="full_pos_swap"></a>
[Back to 5. Run Mutators Tests](#run_mutators_test)<br>

This mutation swaps all players of a randomly selected position between two different teams. It creates more dramatic reconfigurations and fosters diversity at the role level.

**Steps:**

1. Select two different teams.

2. Choose a position (GK, DEF, MID, FWD).

3. Extract all players in that position from both teams.

4. Swap them in bulk.

5. Validate both teams and the league. If invalid, discard mutation.

**Example:**

- Team A: [DEF1, DEF2, ...]

- Team B: [DEF3, DEF4, ...]

- If position = DEF, all defenders from Team A are swapped with those from Team B.

This mutation promotes high-impact changes while preserving internal team structure.

In [10]:
test_mutation(
    population,
    full_position_swap_2teams,
    mutation_name="Full Position Mutation",
    mut_prob=0.5,
    trials=50
)

Full Position Mutation – League #0 over 50 trials:
  no-op (unchanged): 26
  valid mutation: 24

Full Position Mutation – League #1 over 50 trials:
  valid mutation: 25
  no-op (unchanged): 25

Full Position Mutation – League #2 over 50 trials:
  no-op (unchanged): 28
  valid mutation: 20
  invalid (None): 2

Full Position Mutation – League #3 over 50 trials:
  no-op (unchanged): 18
  valid mutation: 32

Full Position Mutation – League #4 over 50 trials:
  valid mutation: 23
  no-op (unchanged): 25
  invalid (None): 2

Full Position Mutation – League #5 over 50 trials:
  valid mutation: 22
  no-op (unchanged): 26
  invalid (None): 2

Full Position Mutation – League #6 over 50 trials:
  valid mutation: 30
  no-op (unchanged): 20

Full Position Mutation – League #7 over 50 trials:
  valid mutation: 26
  no-op (unchanged): 24

Full Position Mutation – League #8 over 50 trials:
  no-op (unchanged): 24
  valid mutation: 23
  invalid (None): 3

Full Position Mutation – League #9 over 50 tria


# 6. Conclusion <a class="anchor" id="conclusion"></a>
[Back to ToC](#toc)<br>

We evaluated the effectiveness of each mutation operator by applying them across a population of 10 leagues, with 50 trials each, totaling 500 mutation attempts per operator. The table below summarizes the results in terms of valid mutations, no-ops (cases where the mutation did not alter the league), and invalid mutations (mutations that resulted in an infeasible solution).

| **Mutator Name**                    | **Function Name**               | **Valid %** | **No-Op %** | **Invalid %** | **Insights**                                                                                   |
| ----------------------------------- | ------------------------------- | ----------- | ----------- | ------------- | ---------------------------------------------------------------------------------------------- |
| Single Player Swap (2 Teams)        | `single_player_swap_2teams`     | 47.4%       | 50.8%       | 1.8%          | Simple mutation with moderate impact. Most failures are due to duplicate players.              |
| Circular Position Shift (All Teams) | `single_player_shift_all_teams` | 48.8%       | 47.6%       | 3.6%          | Produces consistent structural change. Slightly higher failure rate due to strict constraints. |
| Full Position Swap (2 Teams)        | `full_position_swap_2teams`     | 50.0%       | 48.2%       | 1.8%          | Balanced and effective. Swaps full role blocks, maintaining valid structure in most cases.     |
